<h2> Course Project with IOT dataset </h2>

In [1]:
import findspark
import json
findspark.init()
findspark.find()


import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

In [2]:
appName = "Big Data Analytics"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
    .set('spark.driver.host','127.0.0.1')\
    .setAppName(appName)\
    .setMaster(master)

# Create Spark Context with the new configurations rather than relying on the default one
sc = SparkContext.getOrCreate(conf=conf)

# You need to create SQL Context to conduct some database operations like what we will see later.
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
# spark = sqlContext.sparkSession.builder.getOrCreate()
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("GenericAppName") \
    .config('spark.driver.extraClassPath', "C:\spark\jars\postgresql-42.6.0.jar") \
    .config('spark.executor.extraClassPath', "C:\spark\jars\postgresql-42.6.0.jar") \
    .getOrCreate() 

C:\spark\python\pyspark\sql\context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
df_Train = spark.read.csv("train70_reduced.csv",header=True, inferSchema= True)
df_Test = spark.read.csv("test30_reduced.csv",header=True, inferSchema= True)

In [4]:
# Added a new column to both datasets to distunguish between the two
from pyspark.sql.functions import lit
df_Train = df_Train.withColumn('dataset', lit('Train'))
df_Test = df_Test.withColumn('dataset', lit('Test'))

In [5]:
df_Train.show(1, vertical=True)

-RECORD 0--------------------------------
 tcp.flags                  | 0x00000018 
 tcp.time_delta             | 0.998867   
 tcp.len                    | 10         
 mqtt.conack.flags          | 0          
 mqtt.conack.flags.reserved | 0.0        
 mqtt.conack.flags.sp       | 0.0        
 mqtt.conack.val            | 0.0        
 mqtt.conflag.cleansess     | 0.0        
 mqtt.conflag.passwd        | 0.0        
 mqtt.conflag.qos           | 0.0        
 mqtt.conflag.reserved      | 0.0        
 mqtt.conflag.retain        | 0.0        
 mqtt.conflag.uname         | 0.0        
 mqtt.conflag.willflag      | 0.0        
 mqtt.conflags              | 0          
 mqtt.dupflag               | 0.0        
 mqtt.hdrflags              | 0x00000030 
 mqtt.kalive                | 0.0        
 mqtt.len                   | 8.0        
 mqtt.msg                   | 32         
 mqtt.msgid                 | 0.0        
 mqtt.msgtype               | 3.0        
 mqtt.proto_len             | 0.0 

In [6]:
#Rename columns 
for column_name in df_Train.columns:
    new_column_name = column_name.replace(".", "_")
    df_Train = df_Train.withColumnRenamed(column_name, new_column_name)

for column_name in df_Test.columns:
    new_column_name = column_name.replace(".", "_")
    df_Test = df_Test.withColumnRenamed(column_name, new_column_name)

In [7]:
df_Train.show(1, vertical=True)

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000018 
 tcp_time_delta             | 0.998867   
 tcp_len                    | 10         
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0x00000030 
 mqtt_kalive                | 0.0        
 mqtt_len                   | 8.0        
 mqtt_msg                   | 32         
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 3.0        
 mqtt_proto_len             | 0.0 

In [8]:
df_Test.show(1, vertical=True)

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000014 
 tcp_time_delta             | 0.029854   
 tcp_len                    | 0          
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0          
 mqtt_kalive                | 0.0        
 mqtt_len                   | 0.0        
 mqtt_msg                   | 0          
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 0.0        
 mqtt_proto_len             | 0.0 

In [9]:
df_Train.printSchema()

root
 |-- tcp_flags: string (nullable = true)
 |-- tcp_time_delta: double (nullable = true)
 |-- tcp_len: integer (nullable = true)
 |-- mqtt_conack_flags: string (nullable = true)
 |-- mqtt_conack_flags_reserved: double (nullable = true)
 |-- mqtt_conack_flags_sp: double (nullable = true)
 |-- mqtt_conack_val: double (nullable = true)
 |-- mqtt_conflag_cleansess: double (nullable = true)
 |-- mqtt_conflag_passwd: double (nullable = true)
 |-- mqtt_conflag_qos: double (nullable = true)
 |-- mqtt_conflag_reserved: double (nullable = true)
 |-- mqtt_conflag_retain: double (nullable = true)
 |-- mqtt_conflag_uname: double (nullable = true)
 |-- mqtt_conflag_willflag: double (nullable = true)
 |-- mqtt_conflags: string (nullable = true)
 |-- mqtt_dupflag: double (nullable = true)
 |-- mqtt_hdrflags: string (nullable = true)
 |-- mqtt_kalive: double (nullable = true)
 |-- mqtt_len: double (nullable = true)
 |-- mqtt_msg: string (nullable = true)
 |-- mqtt_msgid: double (nullable = true)
 |-

In [10]:
Fake_Kaggle = df_Train.union(df_Test)
Fake_Kaggle.show(1, vertical=True)

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000018 
 tcp_time_delta             | 0.998867   
 tcp_len                    | 10         
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0x00000030 
 mqtt_kalive                | 0.0        
 mqtt_len                   | 8.0        
 mqtt_msg                   | 32         
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 3.0        
 mqtt_proto_len             | 0.0 

In [11]:
#Making sure the databases are combined correctly 
dataset_count = Fake_Kaggle.groupBy('dataset').count()
dataset_count.show()

+-------+------+
|dataset| count|
+-------+------+
|  Train|231646|
|   Test| 99290|
+-------+------+



In [12]:
# Upload KDD Train
db_properties={}
#update your db username
db_properties['username']="postgres"
#update your db password
db_properties['password']="pratik120"
#make sure you got the right port number here
db_properties['url']= "jdbc:postgresql://localhost:5432/postgres"
#make sure you had the Postgres JAR file in the right location
db_properties['driver']="org.postgresql.Driver"
db_properties['table']= "Fake_MQTT"



Fake_Kaggle.write.format("jdbc")\
.mode("overwrite")\
.option("url", db_properties['url'])\
.option("dbtable", db_properties['table'])\
.option("user", db_properties['username'])\
.option("password", db_properties['password'])\
.option("Driver", db_properties['driver'])\
.save()

In [13]:
## Reading the data back
df_read = sqlContext.read.format("jdbc")\
    .option("url", db_properties['url'])\
    .option("dbtable", db_properties['table'])\
    .option("user", db_properties['username'])\
    .option("password", db_properties['password'])\
    .option("Driver", db_properties['driver'])\
    .load()

df_read.show(1, vertical=True)

-RECORD 0--------------------------------
 tcp_flags                  | 0x00000018 
 tcp_time_delta             | 0.998867   
 tcp_len                    | 10         
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_conflags              | 0          
 mqtt_dupflag               | 0.0        
 mqtt_hdrflags              | 0x00000030 
 mqtt_kalive                | 0.0        
 mqtt_len                   | 8.0        
 mqtt_msg                   | 32         
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 3.0        
 mqtt_proto_len             | 0.0 

In [51]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

casted_types_df = df_read.withColumn("casted_mqtt_msg", col("mqtt_msg").cast(DoubleType()))
casted_types_df = casted_types_df.drop("mqtt_msg")
casted_types_df = casted_types_df.withColumnRenamed("casted_mqtt_msg", "mqtt_msg")

In [52]:
casted_types_df.show(2, vertical=True)

-RECORD 0------------------------------------------
 tcp_flags                  | 0x00000018           
 tcp_time_delta             | 0.998867             
 tcp_len                    | 10                   
 mqtt_conack_flags          | 0                    
 mqtt_conack_flags_reserved | 0.0                  
 mqtt_conack_flags_sp       | 0.0                  
 mqtt_conack_val            | 0.0                  
 mqtt_conflag_cleansess     | 0.0                  
 mqtt_conflag_passwd        | 0.0                  
 mqtt_conflag_qos           | 0.0                  
 mqtt_conflag_reserved      | 0.0                  
 mqtt_conflag_retain        | 0.0                  
 mqtt_conflag_uname         | 0.0                  
 mqtt_conflag_willflag      | 0.0                  
 mqtt_conflags              | 0                    
 mqtt_dupflag               | 0.0                  
 mqtt_hdrflags              | 0x00000030           
 mqtt_kalive                | 0.0                  
 mqtt_len   

In [53]:
casted_types_df.select('mqtt_msg').show(2, truncate=False)

+---------------------+
|mqtt_msg             |
+---------------------+
|32.0                 |
|6.361653943666145E199|
+---------------------+
only showing top 2 rows

